In [1]:
import pandas as pd
import re
import time
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from selenium import webdriver
from textacy.preprocess import preprocess_text
import nltk

from bs4 import BeautifulSoup as bs

In [2]:
start = time.time()

wd = webdriver.Chrome()

b_summary = []
b_detail = []
b_date = []
b_summary_clean = []
b_detail_clean = []

for i in range(1,4):
    print 'scrapping page : ', i
    url = 'https://hungrybird.sg/page/' + str(i) + '/'   

    try:
        wd.get(url)
    except:
        continue
    response = wd.page_source
    soup = bs(response, 'html.parser')
    
    try:
        blogs = soup.find_all('h1', {'class' : 'entry-title'})
    except:
        continue
        
    for summary in blogs:
        try:
            href = summary.find('a').get('href')
            print 'href : ', href
        except:
            continue            
        
        try:
            wd.get(href)
        except:
            continue
        
        r = wd.page_source
        s = bs(r, 'html.parser')

        # find date
        try:
            date = s.find('time', {'class' : 'entry-date'}).text
        except:
            continue
            
        # find detail
        try:
            y = s.find('div', {'class' : "entry-content"}).find_all('p')
        except:
            continue
        doc = []
        for p in y:
            if p.text:
                doc.append(p.text)
        d = ' '.join(doc)

        b_summary.append(summary.text)
        b_detail.append(d)
        b_date.append(date)
        
        # clean summary and detail 
        clean_summary = preprocess_text(summary.text, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        clean_detail = preprocess_text(d, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        b_summary_clean.append(clean_summary)
        b_detail_clean.append(clean_detail)
        
        print 'cleaned detail snippet : ', clean_detail[:75]
        
end = time.time()
exe_time = (end - start)/60
print 'Time taken :',(exe_time),' minutes'

print len(b_date)

scrapping page :  1
href :  https://hungrybird.sg/2017/10/03/raffles-singapore-mooncakes-2017-bonus-mooncake-hacks/
cleaned detail snippet :  mid autumn festival is literally around the corner just tomorrow unlike oth
href :  https://hungrybird.sg/2017/09/27/abc-cooking-studio/
cleaned detail snippet :  as much as i love eating out i also love making my own food be it cooking o
href :  https://hungrybird.sg/2017/09/26/shuu-by-koki-dessert/
cleaned detail snippet :  i found the best choux pastry in singapore guys for the longest time i was 
href :  https://hungrybird.sg/2017/09/24/brawn-brains-brunch/
cleaned detail snippet :  im not the sort to cafe hop or have brunch on the regular simply because mo
href :  https://hungrybird.sg/2017/09/23/violet-oon-satay-grill-bar-dinner/
cleaned detail snippet :  ive been to violet oons kitchen bukit timah and national kitchen national g
href :  https://hungrybird.sg/2017/09/17/bao-makers-lunch/
cleaned detail snippet :  i generally do not think mu

In [3]:
# remove stop words from cleaned summary and cleaned detail and lemmatize them

stoplist = set(stopwords.words('english') + \
               ['number',"singapore", "food", "im", "street", 
                'porn','watch', 'video', 'centre', '...',
                'eat', 'day', 'time', 'cdataadsbygoogle', 'windowadsbygoogle',
                'wa', 'ha', 'come', 'place', 'dish', 'bring', 'think', 'quite','located',
                'month', 'went', 'probably', 'am', 'pm', 'say', 'said','including','year','item',
                'youre', 'sure', 'dont', 'came','really', 'got', 'thing', 'address', 'photo',
                'credit', 'opening', 'hour'] \
                 + list(ENGLISH_STOP_WORDS))

In [4]:
start = time.time()

summary_tokens = []
for item in b_summary_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    summary_tokens.append(t)
    
detail_tokens = []
for item in b_detail_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    detail_tokens.append(t)   

    
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'
    
print len(summary_tokens)
print len(detail_tokens)

Time taken : 0.186859846115  seconds
28
28


In [5]:
start = time.time()

summary_lem =[]
wnl = WordNetLemmatizer()
for item in summary_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    summary_lem.append(t) 
    
detail_lem =[]
wnl = WordNetLemmatizer()
for item in detail_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    detail_lem.append(t)     
   
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'

print len(summary_lem)
print len(detail_lem)

Time taken : 3.66977286339  seconds
28
28


In [6]:
# create final list to fit in dataframe
flist = []
for i in range(len(b_summary)):
    f = [b_summary[i], b_detail[i], b_date[i], summary_lem[i], detail_lem[i]]
    flist.append(f)

In [7]:
df = pd.DataFrame(flist, columns = ['Title', 'Details', 'Date', 'Processed Summary', 'Processed Detail'])  
df.tail()

,Title,Details,Date,Processed Summary,Processed Detail
23,Ichigo Daifuku イチゴ大福(Strawberry Mochi) – Recipe,Mochi is a famous wagashi everyone outside of ...,"March 2, 2017",ichigo daifuku ichigoda fu strawberry mochi re...,mochi famous wagashi outside japan know seen r...
24,Ingredients for Wagashi (Japanese Sweets) in S...,So lately I’ve got a new personal project goin...,"March 1, 2017",ingredient wagashi japanese sweet,lately ive new personal project going decided ...
25,Warabimochi わらび餅 – Recipe,When I first sank my teeth into Warabimochi se...,"February 28, 2017",warabimochi warabibing recipe,sank teeth warabimochi year ago fell love moch...
26,Chendol – Recipe,It was during my Yin Yoga Teacher’s Training i...,"February 24, 2017",chendol recipe,yin yoga teacher training jakarta fell love ch...
27,Matcha Azuki Jelly – Recipe,I’ve always loved Japanese wagashi and dessert...,"February 18, 2017",matcha azuki jelly recipe,ive loved japanese wagashi dessert disappointe...


In [8]:
len(df)

28

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
len(df)

28

In [11]:
x = df.iloc[3,1]
print x

I’m not the sort to cafe hop or have brunch on the regular, simply because most of the time, food is usually mediocre and people are just there for the vibes. Just like how coffee people can hang out at coffee places everyday for hours (it is true, I am 100% not exaggerating) because they just like being there. Personally, I cannot resonate to that, hence I seldom ever step into cafes. I don’t even drink coffee (nope, I do not need a daily dose of caffeine to function. I just need food). Among the many black sheep around which serves sub par food, there are of course a few hidden gems which fare better. By that, I mean they put in effort into conceptualizing a proper dish and not just whip up something they copied from others. For that, I will visit cafes and again I will mention, I have no interest in cafe vibes of any sort. Just give me good food and I will go! So I happen to have a weekend intensive yoga workshop this weekend at Yoga Inc, which is located at the old badminton hall o

In [12]:
# pickle dataframe
df.to_pickle('hungrybird.pickle')

In [13]:
wd.quit()